In [105]:
import pandas as pd
import numpy as np
import pyodbc
import math
from ast import literal_eval
from tqdm import tqdm
import os
from configparser import ConfigParser
import statistics
from scipy import stats as s
# from connector.mosaicio import MosaicioConnector

In [106]:
pd.set_option("display.max_column",200)
pd.set_option("display.float_format",lambda x:'%.5f' %x)

In [109]:
data_ind_U  = pd.read_csv(r"/data/Synthetic_data_Clustered_individual_U.csv")
ind_data_thresholds_U = pd.read_csv(r"/data/threshold_individual_U.csv")
weight_table_ind_U = pd.read_csv(r"/data/weight_individual_U.csv")

In [111]:
ind_data_thresholds_U

P003001002  \
0   (None, None, None, None, None, None, None, Non...   
1   (None, None, None, None, None, None, None, Non...   
2   (None, None, None, None, None, None, None, Non...   
3   (None, None, None, None, None, None, None, Non...   
4   (None, None, None, None, None, None, None, Non...   
5   (None, None, None, None, None, None, None, Non...   
6   (None, None, None, None, None, None, None, Non...   
7   (None, None, None, None, None, None, None, Non...   
8   (None, None, None, None, None, None, None, Non...   
9   (None, None, None, None, None, None, None, Non...   
10  (None, None, None, None, None, None, None, Non...   
11  (None, None, None, None, None, None, None, Non...   

                                           P002001024  \
0                      (1, 1, 1, 1, 1, 1, 1, 1, 1, 1)   
1                      (1, 1, 1, 1, 1, 1, 1, 1, 1, 1)   
2                      (2, 3, 3, 4, 4, 5, 5, 6, 6, 7)   
3                      (1, 1, 1, 1, 1, 1, 1, 1, 1, 1)   
4                      (1, 1, 1, 1, 1, 1, 1, 1, 1, 1)   
5                      (1, 1, 1, 1, 1, 1, 1, 1, 1, 1)   
6                      (2, 2, 2, 2, 2, 2, 2, 2, 2, 2)   
7                      (1, 1, 1, 1, 1, 1, 1, 1, 1, 1)   
8   (None, None, None, None, None, None, None, Non...   
9                      (2, 2, 2, 2, 2, 2, 2, 2, 2, 2)   
10                     (1, 1, 1, 1, 1, 1, 1, 1, 1, 1)   
11                     (2, 2, 3, 3, 4, 4, 5, 5, 6, 6)   

                                           P002001001  \
0                      (1, 1, 1, 1, 1, 1, 1, 1, 1, 1)   
1                      (1, 1, 1, 1, 1, 1, 1, 1, 1, 1)   
2                      (1, 2, 2, 3, 3, 4, 4, 4, 5, 5)   
3                      (1, 1, 1, 1, 1, 1, 1, 1, 1, 1)   
4                      (1, 1, 1, 1, 1, 1, 1, 1, 1, 1)   
5                      (1, 1, 1, 1, 1, 1, 1, 1, 1, 1)   
6   (None, None, None, None, None, None, None, Non...   
7                      (1, 1, 1, 1, 1, 1, 1, 1, 1, 1)   
8   (None, None, None, None, None, None, None, Non...   
9   (None, None, None, None, None, None, None, Non...   
10  (None, None, None, None, None, None, None, Non...   
11                     (1, 2, 2, 3, 3, 4, 4, 5, 5, 6)   

                                           P002003001  \
0   (-74, -74, -74, -74, -74, -74, -74, -74, -74, ...   
1   (None, None, None, None, None, None, None, Non...   
2   (None, None, None, None, None, None, None, Non...   
3   (None, None, None, None, None, None, None, Non...   
4   (None, None, None, None, None, None, None, Non...   
5   (None, None, None, None, None, None, None, Non...   
6   (None, None, None, None, None, None, None, Non...   
7   (None, None, None, None, None, None, None, Non...   
8   (None, None, None, None, None, None, None, Non...   
9   (None, None, None, None, None, None, None, Non...   
10  (None, None, None, None, None, None, None, Non...   
11  (None, None, None, None, None, None, None, Non...   

                                           P002001029  \
0   (None, None, None, None, None, None, None, Non...   
1   (None, None, None, None, None, None, None, Non...   
2   (None, None, None, None, None, None, None, Non...   
3   (None, None, None, None, None, None, None, Non...   
4   (None, None, None, None, None, None, None, Non...   
5   (None, None, None, None, None, None, None, Non...   
6   (None, None, None, None, None, None, None, Non...   
7   (None, None, None, None, None, None, None, Non...   
8   (None, None, None, None, None, None, None, Non...   
9   (None, None, None, None, None, None, None, Non...   
10  (None, None, None, None, None, None, None, Non...   
11  (None, None, None, None, None, None, None, Non...   

                                           P002001028  \
0                      (1, 1, 1, 1, 1, 1, 1, 1, 1, 1)   
1   (None, None, None, None, None, None, None, Non...   
2                      (1, 1, 1, 1, 1, 1, 1, 1, 1, 1)   
3   (None, None, None, None, None, None, None, Non...   
4   (None, None, None, None, None, None, None, N

In [112]:
ind_data_thresholds_U['PEER_GROUP_ID'] = ind_data_thresholds_U.apply(lambda row:row['OCCUPATION_BUSINESS_CLEAN']+"_"+str(row['Cluster_labels']),axis=1)
weight_table_ind_U['PEER_GROUP_ID'] = weight_table_ind_U.apply(lambda row:row['OCCUPATION_BUSINESS_CLEAN']+"_"+str(row['Cluster_labels']),axis=1)

In [104]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [116]:
# Changed Code
count = 1
# Populating the threshold values in the threshold table
for ind,row in ind_data_thresholds_U.iterrows():
    if (count%10000 == 0):
        print(count," Done")
    count+=1
    for key in row.keys():
        
        if key.startswith('P0'): #only considering columns which start with P
            
            data = literal_eval(row[key])
#             print(row['PEER_GROUP_ID'],key)
            print("data = ",data)

            if all(elem is None for elem in data) == True:
                print("The rule_no",key,"don't have any value for peer_group",row['PEER_GROUP_ID'])
                conn.execute("""insert into fincore_db.fincore_analytics.RISK_LKP_ENTITY_THRESHOLD (PEER_GROUP_ID,RULE_NO)values(?,?)""",(row['PEER_GROUP_ID'],key))
                print("end")
            else:   
#                 print("data = ",data)
# #                 data = literal_eval(row[key])
                conn.execute("""insert into fincore_db.fincore_analytics.RISK_LKP_ENTITY_THRESHOLD (THRESHOLD1_1,THRESHOLD1_2,THRESHOLD1_3,THRESHOLD1_4,THRESHOLD1_5,THRESHOLD1_6,THRESHOLD1_7,THRESHOLD1_8,THRESHOLD1_9,THRESHOLD1_10,MEAN,MEDIAN,MODE,STANDARD_DEVIATION,OUTLIER,PEER_GROUP_ID,RULE_NO)values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)""",(data[0],data[1],data[2],data[3],data[4],data[5],data[6],data[7],data[8],data[9],data[10],data[11],data[12],data[13],data[14],row['PEER_GROUP_ID'],key))

conn.commit()

data =  (None, None, None, None, None, None, None, None, None, None)
The rule_no P003001002 don't have any value for peer_group Government Sector_0


IntegrityError: ('23000', "[23000] [FreeTDS][SQL Server]Violation of UNIQUE KEY constraint 'UC_RISK_LKP_ENTITY_THRESHOLD'. Cannot insert duplicate key in object 'Fincore_Analytics.RISK_LKP_ENTITY_THRESHOLD'. The duplicate key value is (P003001002, Government Sector_0, <NULL>). (2627) (SQLExecDirectW)")

In [117]:
peers = pd.read_sql_query("Select distinct peer_group_id from fincore_db.fincore_analytics.risk_person_summary_stg with (nolock) ",conn)

In [118]:
peers = peers['peer_group_id'].to_list()

In [119]:
rule_no = pd.read_sql_query("select rule_no from fincore_db.fincore_analytics.risk_lkp_rules with (nolock) where is_active_flag = 1 and RISK_SCORE_STATIC_FLAG = 0 and RULE_NO like 'P%' and risk_type <> 'Auto High Rules '",conn)

In [120]:
rule_no = rule_no['rule_no'].to_list()

In [121]:
data_ind_U.drop_duplicates(inplace = True)

In [122]:
for peer_group_id in peers:
    for rules in rule_no: 
        a = data_ind_U[data_ind_U['PEER_GROUP_ID'] == peer_group_id][rules]
        a = a.dropna()
        a = a.astype('int32')
        if (len(a) == 0):
            th = ['Null','Null']
        else:
            th = []
            th.append(min(a))
            th.append(max(a))
        cursor.execute("""update fincore_db.fincore_analytics.RISK_LKP_ENTITY_THRESHOLD set min = {} , max = {} ,update_date = getdate() where peer_group_id ='{}' and rule_no = '{}' """.format(th[0],th[1],peer_group_id,rules))
        conn.commit()
conn.close()